# Commonsense QA

## Colab setups

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import sys

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/eecs595/commonsense_qa'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)

## Dependency installation.

In [4]:
import json
import codecs
import argparse
from copy import deepcopy
from tqdm import tqdm, trange

import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

Install `datasets`

In [5]:
!pip install datasets
from datasets import load_dataset

Install `transformers`

In [6]:
!pip install transformers
from transformers import (AdamW, get_linear_schedule_with_warmup,
                          BertConfig, BertForMultipleChoice, BertTokenizer,
                          XLNetConfig, XLNetForMultipleChoice, XLNetTokenizer,
                          RobertaConfig, RobertaForMultipleChoice, RobertaTokenizer)

## Utilities

In [7]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def load_model(model='all'):
    if model == 'bert':
        return BertConfig, BertForMultipleChoice, BertTokenizer
    elif model == 'xlnet':
        return XLNetConfig, XLNetForMultipleChoice, XLNetTokenizer
    elif model == 'roberta':
        return RobertaConfig, RobertaForMultipleChoice, RobertaTokenizer
    elif model == 'gpt2':
        raise NotImplemented
    raise NotImplemented


def load_optimizer(args, model, train_size):
    num_training_steps = train_size // args.num_train_epochs
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)],
         'weight_decay': 0.0}
    ]

    optimizer = AdamW(
        optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=num_training_steps)

    return model, optimizer, scheduler


def load_data(dataset='commonsense_qa', preview=-1):

    assert dataset in {'commonsense_qa', 'conv_entail', 'eat'}

    if dataset == 'commonsense_qa':
        ds = load_dataset('commonsense_qa')

        if preview > 0:
            data_tr = ds.data['train']
            question = data_tr['question']
            choices = data_tr['choices']
            answerKey = data_tr['answerKey']
            print(question[preview])
            for label, text in zip(choices[preview]['label'], choices[preview]['text']):
                print(label, text)
            print(answerKey[preview])

    elif dataset == 'conv_entail':
        dev_set = codecs.open('data/conv_entail/dev_set.json', 'r', encoding='utf-8').read()
        act_tag = codecs.open('data/conv_entail/act_tag.json', 'r', encoding='utf-8').read()
        ds = json.loads(dev_set), json.loads(act_tag)

        if preview > 0:
            print('Preview not yet implemented for this dataset.')

    else:
        eat = codecs.open('data/eat/eat_train.json', 'r', encoding='utf-8').read()
        ds = json.loads(eat)

        if preview > 0:
            story = ds[preview]['story']
            label = ds[preview]['label']
            bp = ds[preview]['breakpoint']
            for line in story:
                print(line)
            print(label)
            print(bp)

    return ds

In [8]:
load_data(dataset='commonsense_qa', preview=5)

Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)


What home entertainment equipment requires cable?
A radio shack
B substation
C cabinet
D television
E desk
D


DatasetDict({'train': Dataset(features: {'answerKey': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'choices': Sequence(feature={'label': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}, length=-1, id=None)}, num_rows: 9741), 'validation': Dataset(features: {'answerKey': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'choices': Sequence(feature={'label': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}, length=-1, id=None)}, num_rows: 1221), 'test': Dataset(features: {'answerKey': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'choices': Sequence(feature={'label': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}, length=-1, id=None)}, num_rows: 1140)})

## Data Preprocessing

In [9]:
class InputExample(object):
    """
    A single multiple choice question.
    """

    def __init__(self, example_id, question, answers, label):
        self.example_id = example_id
        self.question = question
        self.answers = answers
        self.label = label


class InputFeatures(object):
    """
    A single feature converted from an example.
    """

    def __init__(self, example_id, choices_features, label):
        self.example_id = example_id
        self.label = label
        self.choices_features = [
            {'input_ids': input_ids, 'input_mask': input_mask, 'segment_ids': segment_ids}
            for _, input_ids, input_mask, segment_ids in choices_features
        ]


class CommonsenseQAProcessor:
    """
    A Commonsense QA Data Processor
    """

    def __init__(self):
        self.dataset = None
        self.labels = [0, 1, 2, 3, 4]
        self.LABELS = ['A', 'B', 'C', 'D', 'E']

    def get_split(self, split='train'):
        if self.dataset is None:
            self.dataset = load_data(dataset='commonsense_qa', preview=-1)
        return self.dataset[split]

    def create_examples(self, split='train'):
        examples = []
        data_tr = self.get_split(split)
        example_id = 0

        for question, choices, answerKey in zip(data_tr['question'], data_tr['choices'], data_tr['answerKey']):
            answers = np.array(choices['text'])
            label = self.LABELS.index(answerKey)
            examples.append(InputExample(
                example_id=example_id, question=question,
                answers=answers, label=label
            ))
            example_id += 1

        return examples


def truncate_seq_pair(tokens_a, tokens_b, max_length):
    """
    Truncates a sequence pair in place to the maximum length.

    This is a simple heuristic which will always truncate the longer sequence one token at a time.
    This makes more sense than truncating an equal percent of tokens from each,
    since if one sequence is very short then each token that's truncated
    likely contains more information than a longer sequence.

    However, since we'd better not to remove tokens of options and questions,
    you can choose to use a bigger length or only pop from context
    """

    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            warning = 'Attention! you are removing from token_b (swag task is ok). ' \
                      'If you are training ARC and RACE (you are popping question + options), ' \
                      'you need to try to use a bigger max seq length!'
            print(warning)
            tokens_b.pop()


def examples_to_features(examples, label_list, max_seq_length, tokenizer,
                         cls_token_at_end=False,
                         cls_token='[CLS]',
                         cls_token_segment_id=1,
                         sep_token='[SEP]',
                         sequence_a_segment_id=0,
                         sequence_b_segment_id=1,
                         sep_token_extra=False,
                         pad_token_segment_id=0,
                         pad_on_left=False,
                         pad_token=0,
                         mask_padding_with_zero=True):
    """
    Convert Commonsense QA examples to features.

    The convention in BERT is:
    (a) For sequence pairs:
    tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1

    (b) For single sequences:
    tokens:   [CLS] the dog is hairy . [SEP]
    type_ids:   0   0   0   0  0     0   0

    Where "type_ids" are used to indicate whether this is the first sequence or the second sequence.
    The embedding vectors for `type=0` and `type=1` were learned during pre-training
    and are added to the word piece embedding vector (and position vector).
    This is not *strictly* necessary since the [SEP] token unambiguously separates the sequences,
    but it makes it easier for the model to learn the concept of sequences.

    For classification tasks, the first vector (corresponding to [CLS]) is used as as the "sentence vector".
    Note that this only makes sense because the entire model is fine-tuned.
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in tqdm(enumerate(examples), desc="Converting examples to features", disable=True):

        choices_features = []
        for ending_idx, (question, answers) in enumerate(zip(example.question, example.answers)):

            tokens_a = tokenizer.tokenize(example.question)
            if example.question.find("_") != -1:
                tokens_b = tokenizer.tokenize(example.question.replace("_", answers))
            else:
                tokens_b = tokenizer.tokenize(answers)

            special_tokens_count = 4 if sep_token_extra else 3
            truncate_seq_pair(tokens_a, tokens_b, max_seq_length - special_tokens_count)

            tokens = tokens_a + [sep_token]
            if sep_token_extra:
                tokens += [sep_token]

            segment_ids = [sequence_a_segment_id] * len(tokens)

            if tokens_b:
                tokens += tokens_b + [sep_token]
                segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

            if cls_token_at_end:
                tokens = tokens + [cls_token]
                segment_ids = segment_ids + [cls_token_segment_id]
            else:
                tokens = [cls_token] + tokens
                segment_ids = [cls_token_segment_id] + segment_ids

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens.
            # Only real tokens are attended to.
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding_length = max_seq_length - len(input_ids)

            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids

            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = label_map[example.label]

        if ex_index < 0:
            print("*** Example ***")
            print("race_id: {}".format(example.example_id))
            for choice_idx, (tokens, input_ids, input_mask, segment_ids) in enumerate(choices_features):
                print("choice: {}".format(choice_idx))
                print("tokens: {}".format(' '.join(tokens)))
                print("input_ids: {}".format(' '.join(map(str, input_ids))))
                print("input_mask: {}".format(' '.join(map(str, input_mask))))
                print("segment_ids: {}".format(' '.join(map(str, segment_ids))))
                print("label: {}".format(label))

        features.append(InputFeatures(
            example_id=example.example_id,
            choices_features=choices_features,
            label=label
        ))

    return features


def load_features(args, tokenizer, mode='train'):
    """
    Load the processed Commonsense QA dataset
    """

    def select_field(feature_list, field_name):
        return [
            [choice[field_name] for choice in feature.choices_features]
            for feature in feature_list
        ]

    assert mode in {'train', 'validation', 'test'}
    print("Creating features from dataset...")

    processor = CommonsenseQAProcessor()
    label_list = processor.labels
    examples = processor.create_examples(split=mode)

    print("Training number:", str(len(examples)))
    features = examples_to_features(examples, label_list, args.max_seq_length, tokenizer,
                                    cls_token_at_end=bool(args.model_type in ['xlnet']),
                                    cls_token=tokenizer.cls_token,
                                    sep_token=tokenizer.sep_token,
                                    sep_token_extra=bool(args.model_type in ['roberta']),
                                    cls_token_segment_id=2 if args.model_type in ['xlnet'] else 0,
                                    pad_on_left=bool(args.model_type in ['xlnet']),
                                    pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(select_field(features, 'input_ids'), dtype=torch.long)
    all_input_mask = torch.tensor(select_field(features, 'input_mask'), dtype=torch.long)
    all_segment_ids = torch.tensor(select_field(features, 'segment_ids'), dtype=torch.long)
    all_label_ids = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

## Training, Validation and Testing

In [19]:
def train(args, model, tokenizer):

    print('\n Loading training dataset')
    dataset_tr = load_features(args, tokenizer, mode='train')
    sampler_tr = RandomSampler(dataset_tr)
    dataloader_tr = DataLoader(dataset_tr, sampler=sampler_tr, batch_size=args.batch_size)

    print('\n Loading validation dataset')
    dataset_val = load_features(args, tokenizer, 'validation')
    sampler_val = SequentialSampler(dataset_val)
    dataloader_val = DataLoader(dataset_val, sampler=sampler_val, batch_size=args.batch_size)

    model, optimizer, scheduler = load_optimizer(args, model, len(dataloader_tr))

    num_steps = 0
    best_steps = 0
    tr_loss = 0.0
    best_val_acc, best_val_loss = 0.0, 99999999999.0
    best_model = None

    model.zero_grad()
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=False, leave=True, position=1)

    for _ in train_iterator:

        epoch_iterator = tqdm(dataloader_tr, desc="Iteration", disable=False, leave=True, position=1)
        for step, batch in enumerate(epoch_iterator):

            model.train()

            batch = tuple(b.to(args.device) for b in batch)
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args.model_type in ['bert', 'xlnet'] else None,
                      'labels': batch[3]}
            outputs = model(**inputs)
            loss = outputs[0]

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()

            optimizer.step()
            scheduler.step()
            model.zero_grad()
            num_steps += 1

            if args.logging_steps > 0 and num_steps % args.logging_steps == 0:
                results = evaluate(args, model, dataloader_val)
                print("\n val acc: {}, val loss: {}"
                      .format(str(results['val_acc']), str(results['val_loss'])))
                if results["val_acc"] > best_val_acc:
                    best_val_acc, best_val_loss = results["val_acc"], results["val_loss"]
                    best_steps = num_steps
                    best_model = deepcopy(model)

    loss = tr_loss / num_steps

    return best_model


def evaluate(args, model, dataloader):

    val_loss = 0.0
    num_steps = 0
    preds, labels = None, None

    results = {}

    for batch in tqdm(dataloader, desc="Validation", disable=True, leave=True, position=1):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args.model_type in ['bert', 'xlnet'] else None,
                      'labels': batch[3]}
            outputs = model(**inputs)
            loss, logits = outputs[:2]

            val_loss += loss.mean().item()

        num_steps += 1

        if preds is None:
            preds = logits.detach().cpu().numpy()
            labels = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            labels = np.append(labels, inputs['labels'].detach().cpu().numpy(), axis=0)

    loss = val_loss / num_steps
    preds = np.argmax(preds, axis=1)
    acc = (preds == labels).mean()
    result = {"val_acc": acc, "val_loss": loss}
    results.update(result)

    return results


def test(args, model):
    dataset = load_features(args, tokenizer, mode='test')
    sampler = RandomSampler(dataset)
    dataloader_tr = DataLoader(dataset, sampler=sampler, batch_size=args.batch_size)
    results = evaluate(args, model, dataloader_val)
    print("test acc: %s, test loss: %s",
          str(results['val_acc']), str(results['val_loss']))

## Run experiment

In [20]:
def main(args):

    print('Using device', args.device)
    set_seed(args.seed)

    if args.mode == 'train':
        processor = CommonsenseQAProcessor()
        num_labels = len(processor.labels)

        config_class, model_class, tokenizer_class = load_model(args.model_type)
        config = config_class.from_pretrained(
            args.config_name if args.config_name else args.model_name,
            num_labels=num_labels, finetuning_task=args.task_name)
        tokenizer = tokenizer_class.from_pretrained(
            args.tokenizer_name if args.tokenizer_name else args.model_name,
            do_lower_case=True)
        model = model_class.from_pretrained(
            args.model_name, from_tf=bool('.ckpt' in args.model_name), config=config)

        model.to(args.device)

        best_model = train(args, model, tokenizer)

        test(args, best_model)


if __name__ == "__main__":

    parser = argparse.ArgumentParser(description="Common sense question answering")

    parser.add_argument("--mode", type=str, default='train',
                        help="Mode: <str> [ train | test ]")
    parser.add_argument("--model_type", type=str, default='bert',
                        help="Model: <str> [ bert | xlnet | roberta | gpt2 ]")
    parser.add_argument("--task_name", default=None, type=str, required=False,
                        help="The name of the task to train: <str> [ commonqa ]")
    parser.add_argument("--model_name", type=str,
                        default='bert-base-uncased',
                        help="Path to pre-trained model or shortcut name."
                             "See https://huggingface.co/models")
    parser.add_argument("--config_name", type=str,
                        default='bert-base-uncased',
                        help="Pre-trained config name or path")
    parser.add_argument("--tokenizer_name", default='bert-base-uncased', type=str,
                        help="Pre-trained tokenizer name or path if not the same as model_name")

    parser.add_argument("--max_seq_length", default=128, type=int,
                        help="The maximum total input sequence length after tokenization. "
                             "Sequences longer than this will be truncated, sequences shorter will be padded.")
    parser.add_argument("--batch_size", default=8, type=int,
                        help="Batch size for training.")

    parser.add_argument("--learning_rate", default=5e-5, type=float,
                        help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")

    parser.add_argument("--num_train_epochs", default=3.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--warmup_steps", default=0, type=int,
                        help="Linear warmup over warmup_steps.")
    parser.add_argument('--logging_steps', type=int, default=5,
                        help="Log every n updates steps.")

    parser.add_argument('--fp16', action='store_true',
                        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit")
    parser.add_argument('--opt_level', type=str, default='O1',
                        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
                             "See details at https://nvidia.github.io/apex/amp.html")

    parser.add_argument("--seed", type=int, default=0, help="Random seed: <int>")
    parser.add_argument("--device", default=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    args, unknown = parser.parse_known_args()
    main(args)


Using device cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly


 Loading training dataset
Creating features from dataset...
Training number: 9741


Using custom data configuration default
Reusing dataset commonsense_qa (/root/.cache/huggingface/datasets/commonsense_qa/default/0.1.0/0e60f0ee8c8509e854ed897f65eb5b2e6ca22578d64cbc3812c79b527d7a7a29)



 Loading validation dataset
Creating features from dataset...
Training number: 1221


...


Iteration:   0%|          | 6/1218 [04:44<16:03:54, 47.72s/it]


 val acc: 0.32432432432432434, val loss: 1.578283889620912



Iteration:   1%|          | 7/1218 [05:33<16:14:08, 48.26s/it]


 val acc: 0.31695331695331697, val loss: 1.5731143468345692


KeyboardInterrupt: ignored

## Todo

The current model is based on `bert`.
```
    parser.add_argument("--model_name", type=str,
                        default='bert-base-uncased',
                        help="Path to pre-trained model or shortcut name."
                             "See https://huggingface.co/models")
```

This would leads to

```
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using custom data configuration default
```

Should check https://huggingface.co/models for other models.